<a href="https://colab.research.google.com/github/VOTUANANH01/DS104_Distributed_and_parallel_computing/blob/main/Slide%205-6/Tutorial%205/Tutorial_5_Test_Kafka_and_Spark_Structure_Streaming_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/3.3.1/kafka_2.13-3.3.1.tgz
!tar -xzf kafka_2.13-3.3.1.tgz

In [ ]:
!./kafka_2.13-3.3.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.3.1/config/zookeeper.properties
!./kafka_2.13-3.3.1/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.3.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!pip install pyspark==3.2.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 50.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5c9733fd07309dc5ad96e7a4e4a445a5126a3d9b14d2f71acc8722bb840ca172
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
scala_version = '2.12'
spark_version = '3.2.1'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}' , 'org.apache.kafka:kafka-clients:3.3.1' ]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [ ]:
!pip install kafka-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 5.2 MB/s 


In [ ]:
from kafka import KafkaProducer
from json import dumps
topic_name = 'Number'
kafka_server = '127.0.0.1:9092'
producer = KafkaProducer(bootstrap_servers=kafka_server,value_serializer = lambda x:dumps(x).encode('utf-8'))
for e in range(1000):
  data = {'number' : e}
  producer.send(topic_name, value=data)
producer.flush()

In [ ]:
!./kafka_2.13-3.3.1/bin/kafka-console-consumer.sh --topic Number --from-beginning --bootstrap-server 127.0.0.1:9092

{"number": 0}
{"number": 1}
{"number": 2}
{"number": 3}
{"number": 4}
{"number": 5}
{"number": 6}
{"number": 7}
{"number": 8}
{"number": 9}
{"number": 10}
{"number": 11}
{"number": 12}
{"number": 13}
{"number": 14}
{"number": 15}
{"number": 16}
{"number": 17}
{"number": 18}
{"number": 19}
{"number": 20}
{"number": 21}
{"number": 22}
{"number": 23}
{"number": 24}
{"number": 25}
{"number": 26}
{"number": 27}
{"number": 28}
{"number": 29}
{"number": 30}
{"number": 31}
{"number": 32}
{"number": 33}
{"number": 34}
{"number": 35}
{"number": 36}
{"number": 37}
{"number": 38}
{"number": 39}
{"number": 40}
{"number": 41}
{"number": 42}
{"number": 43}
{"number": 44}
{"number": 45}
{"number": 46}
{"number": 47}
{"number": 48}
{"number": 49}
{"number": 50}
{"number": 51}
{"number": 52}
{"number": 53}
{"number": 54}
{"number": 55}
{"number": 56}
{"number": 57}
{"number": 58}
{"number": 59}
{"number": 60}
{"number": 61}
{"number": 62}
{"number": 63}
{"number": 64}
{"number": 65}
{"number": 66}
{"num

In [ ]:
kafkaDf = spark.read.format("kafka")\
  .option("kafka.bootstrap.servers", "127.0.0.1:9092")\
  .option("subscribe", topic_name)\
  .option("startingOffsets", "earliest")\
  .load()
kafkaDf.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 6E 75 6D 6...|Number|        0|     0|2022-10-11 08:02:...|            0|
|null|[7B 22 6E 75 6D 6...|Number|        0|     1|2022-10-11 08:02:...|            0|
|null|[7B 22 6E 75 6D 6...|Number|        0|     2|2022-10-11 08:02:...|            0|
|null|[7B 22 6E 75 6D 6...|Number|        0|     3|2022-10-11 08:02:...|            0|
|null|[7B 22 6E 75 6D 6...|Number|        0|     4|2022-10-11 08:02:...|            0|
|null|[7B 22 6E 75 6D 6...|Number|        0|     5|2022-10-11 08:02:...|            0|
|null|[7B 22 6E 75 6D 6...|Number|        0|     6|2022-10-11 08:02:...|            0|
|null|[7B 22 6E 75 6D 6...|Number|        0|     7|2022-10-11 08:02:...|            0|
|null|[7B 22 6E 75 6D 6...|Number|        0

In [ ]:
from pyspark.sql.functions import col, concat, lit
kafkaDf.select(
concat(col("topic"), lit(':'),
col("partition").cast("string")).alias("topic_partition"),col("offset"),col("value").cast("string")
).show()

+---------------+------+--------------+
|topic_partition|offset|         value|
+---------------+------+--------------+
|       Number:0|     0| {"number": 0}|
|       Number:0|     1| {"number": 1}|
|       Number:0|     2| {"number": 2}|
|       Number:0|     3| {"number": 3}|
|       Number:0|     4| {"number": 4}|
|       Number:0|     5| {"number": 5}|
|       Number:0|     6| {"number": 6}|
|       Number:0|     7| {"number": 7}|
|       Number:0|     8| {"number": 8}|
|       Number:0|     9| {"number": 9}|
|       Number:0|    10|{"number": 10}|
|       Number:0|    11|{"number": 11}|
|       Number:0|    12|{"number": 12}|
|       Number:0|    13|{"number": 13}|
|       Number:0|    14|{"number": 14}|
|       Number:0|    15|{"number": 15}|
|       Number:0|    16|{"number": 16}|
|       Number:0|    17|{"number": 17}|
|       Number:0|    18|{"number": 18}|
|       Number:0|    19|{"number": 19}|
+---------------+------+--------------+
only showing top 20 rows

